# Baltimore daily shootings analysis

By [Christine Zhang](mailto:czhang@baltsun.com)

The Baltimore Sun conducted the following analysis of Baltimore Police Department Part 1 Victim Based Crime Data posted on [Open Baltimore](https://data.baltimorecity.gov/Public-Safety/BPD-Part-1-Victim-Based-Crime-Data/wsfq-mvij) on Feb. 22, 2019. The report covers incidents between Jan. 1, 2012, and Feb. 16, 2019.

The analysis provided information for the Feb. 22, 2019 Baltimore Sun story titled ["Thursday's violence in Baltimore was extreme — but the number of shootings isn't unprecedented"](https://www.baltimoresun.com/news/maryland/crime/bs-md-ci-homicide-record-numbers-20190222-story.html).

The main finding from the analysis presented in the story was that Feb. 21, 2019 tied with at least two other dates believed to be among the highest documented number of shootings in single day:

- In May of 2015, 12 people were shot, three fatally, in a single day.
- In Sept. 2016, 12 people were shot, one fatally, on a Saturday.
- Eight people struck by gunfire at the same time, date and address in Sept. 2016

## Import R data analysis libraries

In [1]:
suppressWarnings(suppressMessages(library('tidyverse')))
suppressWarnings(suppressMessages(library('lubridate')))

## Data processing:

### Read in BPD Part 1 Victim Based Crime Data for analysis

In [2]:
crime <- read.csv("BPD_Part_1_Victim_Based_Crime_Data.csv", stringsAsFactors = F)

Create a column, `date`, which formats the `CrimeDate` field as a date. Create a new dataframe, `shootings`, which filter the datas to include shootings and homicides for which a firearm was the weapon used. Also create a column, `fatal` which indicates whether or not the shooting was fatal (ie.gl, a homicide with a firearm used as a weapon).

In [3]:
crime$date <- mdy(crime$CrimeDate)
shootings <- crime %>% filter((Description == 'SHOOTING' | Description == 'HOMICIDE') & Weapon == 'FIREARM') %>%
                       arrange(date) %>% mutate(fatal = ifelse(Description == 'HOMICIDE', 1, 0))

## Data analysis:

### Finding: In May of 2015, 12 people were shot, three fatally, in a single day.

Create a dataframe, `shootings.bydate`, which groups the shootings by date and sums the total incidents (`n`) and fatal incidents (`fatal_n`). Create a column, `dow`, which indicates the day of week. Arrange `shootings.bydate` by date in descending order of `n`.

In [4]:
shootings.bydate <- shootings %>% 
  group_by(date) %>% 
  summarise(n = sum(Total.Incidents, na.rm = T),
            fatal_n = sum(fatal, na.rm = T)) %>% 
  mutate(dow = wday(date, label=TRUE)) %>% 
  arrange(-n)

In [5]:
shootings.bydate %>% head()

date,n,fatal_n,dow
2015-05-20,12,3,Wed
2016-09-24,12,1,Sat
2015-07-12,11,4,Sun
2016-12-20,11,2,Tue
2018-06-30,11,0,Sat
2018-07-21,11,3,Sat


In [6]:
print(paste("On", 
             shootings.bydate[1,]$date, ":",
             shootings.bydate[1,]$n, 
             "shootings were recorded; of those,",
             shootings.bydate[1,]$fatal_n,
             "were fatal."))

[1] "On 2015-05-20 : 12 shootings were recorded; of those, 3 were fatal."


### Finding: In Sept. 2016, 12 people were shot, one fatally, on a Saturday.

In [7]:
print(paste("On", 
             shootings.bydate[2,]$dow,
             shootings.bydate[2,]$date, ":",
             shootings.bydate[2,]$n, 
             "shootings were recorded; of those,",
             shootings.bydate[2,]$fatal_n,
             "was fatal."))

[1] "On Sat 2016-09-24 : 12 shootings were recorded; of those, 1 was fatal."


### Finding: Eight people struck by gunfire at the same time, date and address in Sept. 2016

Group `shootings` by `Location`, `CrimeTime` and `date` and sum the total incidents (`n`). Arrange in descending order of `n`.

In [8]:
shootings %>% 
    group_by(Location, CrimeTime, date) %>% 
    summarise(n = sum(Total.Incidents, na.rm = T)) %>% 
    arrange(-n) %>% head() 

Location,CrimeTime,date,n
700 E PRESTON ST,2029,2016-09-24,8
1500 W FAYETTE ST,2115,2013-08-24,7
2800 W GARRISON AV,0242,2015-08-02,7
2900 GARRISON BLVD,1800,2016-11-30,6
4000 PENHURST AV,2344,2018-06-30,6
1600 E LANVALE ST,1637,2015-05-16,5


There were 8 shootings recorded at the same time, date and address on Sept. 24, 2016. We can filter `shootings` to view these in the context of the 12 total shootings that were recorded on that date.

In [9]:
shootings %>% filter(date == '2016-09-24') %>% 
              arrange(Location, CrimeTime) 

CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside.Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location.1,Premise,crimeCaseNumber,Total.Incidents,date,fatal
09/24/2016,1309,1F,0 S CAREY ST,HOMICIDE,Outside,FIREARM,931,WESTERN,Hollins Market,-76.63811,39.28851,"(39.288510000000, -76.638110000000)",,NA,1,2016-09-24,1
09/24/2016,1833,9S,600 S SMALLWOOD ST,SHOOTING,Outside,FIREARM,841,SOUTHWESTERN,Carrollton Ridge,-76.65033,39.28033,"(39.280330000000, -76.650330000000)",Street,NA,1,2016-09-24,0
09/24/2016,1833,9S,600 S SMALLWOOD ST,SHOOTING,Outside,FIREARM,841,SOUTHWESTERN,Carrollton Ridge,-76.65033,39.28033,"(39.280330000000, -76.650330000000)",Street,NA,1,2016-09-24,0
09/24/2016,1833,9S,600 S SMALLWOOD ST,SHOOTING,Outside,FIREARM,841,SOUTHWESTERN,Carrollton Ridge,-76.65033,39.28033,"(39.280330000000, -76.650330000000)",Street,NA,1,2016-09-24,0
09/24/2016,2029,9S,700 E PRESTON ST,SHOOTING,Outside,FIREARM,311,EASTERN,Johnston Square,-76.60810,39.30485,"(39.304850000000, -76.608100000000)",Street,NA,1,2016-09-24,0
09/24/2016,2029,9S,700 E PRESTON ST,SHOOTING,Outside,FIREARM,311,EASTERN,Johnston Square,-76.60810,39.30485,"(39.304850000000, -76.608100000000)",Street,NA,1,2016-09-24,0
09/24/2016,2029,9S,700 E PRESTON ST,SHOOTING,Outside,FIREARM,311,EASTERN,Johnston Square,-76.60810,39.30485,"(39.304850000000, -76.608100000000)",Street,NA,1,2016-09-24,0
09/24/2016,2029,9S,700 E PRESTON ST,SHOOTING,Outside,FIREARM,311,EASTERN,Johnston Square,-76.60810,39.30485,"(39.304850000000, -76.608100000000)",Street,NA,1,2016-09-24,0
09/24/2016,2029,9S,700 E PRESTON ST,SHOOTING,Outside,FIREARM,311,EASTERN,Johnston Square,-76.60810,39.30485,"(39.304850000000, -76.608100000000)",Street,NA,1,2016-09-24,0
09/24/2016,2029,9S,700 E PRESTON ST,SHOOTING,Outside,FIREARM,311,EASTERN,Johnston Square,-76.60810,39.30485,"(39.304850000000, -76.608100000000)",Street,NA,1,2016-09-24,0
